In [1]:
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [32]:
M = 101

### Returns prob index
def map_value_to_vir_index(x):
  return x*(M-1) + 1

def map_vir_index_to_val(prob_x):
  return (prob_x - 1) / (M - 1)

def solve_copula(minimize = True):
  C = cp.Variable((M, M))

  # Prelogue constraint
  constraints = [
      C[:, map_value_to_vir_index(0) - 1] == 0,
      C[map_value_to_vir_index(0) - 1, :] == 0,
      C[:, map_value_to_vir_index(1) - 1] == [map_vir_index_to_val(i) for i in np.arange(1, M + 1)],
      C[map_value_to_vir_index(1) - 1, :] == [map_vir_index_to_val(i) for i in np.arange(1, M + 1)]
  ]


  for i in np.arange(1, M+1):
    
    #1st constraint
    constraints += [ C[M - 1, i-1] == ((i - 1) / (M - 1)) ]
    constraints += [ C[i -1, M - 1] == ((i - 1) / (M - 1))]

    #2nd constraint
    constraints += [ C[0, i -1] == 0 ]
    constraints += [ C[i -1, 0] == 0 ]

  # 3rd constraint
  for i in np.arange(1, M):
    for j in np.arange(1, M):
      constraints += [ 
          C[i, j] - C[i-1, j] - C[i, j-1] + C[i-1, j-1] >= 0
      ]


  # 4th constraint
  constraints += [
      C[map_value_to_vir_index(0.5) - 1, map_value_to_vir_index(0.9) - 1] == (0.5 * 0.9),
      C[map_value_to_vir_index(0.1) - 1, map_value_to_vir_index(0.6) - 1] == (0.1 * 0.6),
      C[map_value_to_vir_index(0.3) - 1, map_value_to_vir_index(0.3) - 1] == (0.3 * 0.3),
      C[map_value_to_vir_index(0.6) - 1, map_value_to_vir_index(0.2) - 1] == (0.6 * 0.2), 
  ]

  # 5th constraint
  for i in np.arange(1, M+1):
    for j in np.arange(1, M+1):
      constraints += [ 
          C[i-1, j-1] >= 0,
          C[i-1, j-1] <= 1,
      ]

  if minimize:
    problem = cp.Problem(cp.Minimize(C[50][50]), constraints)
  else:
    problem = cp.Problem(cp.Maximize(C[50][50]), constraints)
  
  return problem.solve()

In [33]:
min_value = solve_copula()
min_value

In [34]:
max_value = solve_copula(minimize = False)
max_value

In [36]:
print(f"Min/Max: ({min_value}, {max_value})")

Min/Max: (0.08999999997135398, 0.4200000000011655)
